<a href="https://colab.research.google.com/github/pcummer/deep_learning_short_projects/blob/main/Protein_folding_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Very initial work**

This is an exploratory project to play around with applying various NLP techniques to predict the secondary structure of a protein from the amino acid sequence. Right now, I just have the foundation of loading the data and training a toy model. There's a lot of directions to take this with likely highlights being deploying a proper NLP model like a biLSTM with randomly initialized embeddings for the amino acids and incorporating biophysical properties of the amino acids such as electronegativity and flags for special amino acids such as proline.

In [92]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/protein-secondary-structure/protein-secondary-structure.train
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/protein-secondary-structure/protein-secondary-structure.test

--2020-11-13 16:13:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/protein-secondary-structure/protein-secondary-structure.train
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73489 (72K) [application/x-httpd-php]
Saving to: ‘protein-secondary-structure.train.1’

protein-secondary-s 100%[===================>]  71.77K  --.-KB/s    in 0.1s    

2020-11-13 16:13:58 (686 KB/s) - ‘protein-secondary-structure.train.1’ saved [73489/73489]

--2020-11-13 16:13:58--  https://archive.ics.uci.edu/ml/machine-learning-databases/molecular-biology/protein-secondary-structure/protein-secondary-structure.test
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response...

In [93]:
import pandas as pd
import numpy as np

def parse_to_df(path):
  with open(path,'r') as f:
    content = f.readlines()
  df = pd.DataFrame()
  protein_count = 0
  amino_acids = []
  structure = []
  for i in content:
    i = i.strip()
    if 'end' in i:
        df = df.append(pd.DataFrame({'amino_acid':[amino_acids], 'structure':[structure], 'protein_count':[protein_count]}))
        protein_count += 1
        amino_acids = []
        structure = []
    elif len(i) == 3:
      amino_acids.append(i.split(' ')[0])
      structure.append(i.split(' ')[1])
  return df

Here we load the text files for the train and test splits into easily accessed dataframes. We also also assign an index to each amino acid and structure to replace the text character.

In [94]:
df_train_raw = parse_to_df('/content/protein-secondary-structure.train')
df_test_raw = parse_to_df('/content/protein-secondary-structure.test')


unique_amino_acids_in_train = np.unique([item for sublist in df_train_raw.amino_acid for item in sublist])
unique_amino_acids_in_test = np.unique([item for sublist in df_test_raw.amino_acid for item in sublist])
[unique_amino_acids_in_train.append(x) for x in unique_amino_acids_in_test if x not in unique_amino_acids_in_train]

amino_acid_to_index = {}
i=0
for x in unique_amino_acids_in_train:
  i += 1
  amino_acid_to_index[x] = i

structure_to_index = {'_': 0, 'h': 1, 'e': 2}

df_train_raw['amino_acid_index'] = [[amino_acid_to_index[x] for x in y] for y in df_train_raw.amino_acid]
df_test_raw['amino_acid_index'] = [[amino_acid_to_index[x] for x in y] for y in df_test_raw.amino_acid]
df_train_raw['structure_index'] = [[structure_to_index[x] for x in y] for y in df_train_raw.structure]
df_test_raw['structure_index'] = [[structure_to_index[x] for x in y] for y in df_test_raw.structure]

In [95]:
class basicGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, shuffle=True, batch_size=1):
      self.df = df
      self.shuffle = shuffle
      self.batch_size = batch_size
      self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1.0)

    def __getitem__(self, index):
        indexes = np.arange(index * self.batch_size, (index + 1) * self.batch_size)
        batch_input = []
        batch_target = []
        for i in indexes:
            amino_acid_sequence = self.df.amino_acid_index.iloc[i]
            label_sequence = self.df.structure_index.iloc[i]
            batch_input.append(amino_acid_sequence)
            batch_target.append(label_sequence)

        batch_input = np.stack(batch_input)
        batch_target = np.array(batch_target)

        return batch_input, batch_target


In [96]:
train_generator = basicGenerator(df_train_raw)
test_generator = basicGenerator(df_test_raw)

Here we test a toy model to confirm that our data loading and formatting has worked as expected.

In [113]:
import tensorflow as tf

input = tf.keras.layers.Input((None, 1))
output = tf.keras.layers.LSTM(3, return_sequences=True, activation='softmax')(input)

toy_model = tf.keras.models.Model(inputs=[input], outputs=[output])
toy_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [114]:
toy_model.fit(train_generator, validation_data=test_generator, epochs=10)

Epoch 1/10
91/91 [==============================] - 6s 61ms/step - loss: 4.9527 - acc: 0.2541 - val_loss: 3.6644 - val_acc: 0.2412
Epoch 2/10
91/91 [==============================] - 5s 57ms/step - loss: 2.4639 - acc: 0.2567 - val_loss: 1.7367 - val_acc: 0.2472
Epoch 3/10
91/91 [==============================] - 5s 55ms/step - loss: 1.2891 - acc: 0.3349 - val_loss: 1.1599 - val_acc: 0.4142
Epoch 4/10
91/91 [==============================] - 4s 48ms/step - loss: 1.0809 - acc: 0.4567 - val_loss: 1.0843 - val_acc: 0.4830
Epoch 5/10
91/91 [==============================] - 4s 46ms/step - loss: 1.0489 - acc: 0.4991 - val_loss: 1.0601 - val_acc: 0.5162
Epoch 6/10
91/91 [==============================] - 4s 48ms/step - loss: 1.0357 - acc: 0.5104 - val_loss: 1.0478 - val_acc: 0.5224
Epoch 7/10
91/91 [==============================] - 4s 49ms/step - loss: 1.0267 - acc: 0.5191 - val_loss: 1.0408 - val_acc: 0.5233
Epoch 8/10
91/91 [==============================] - 4s 49ms/step - loss: 1.0197 - a

In [115]:
from sklearn.metrics import classification_report

predictions = [toy_model.predict(x) for x in test_generator]
print(classification_report([z for _, y in test_generator for x in y for z in x], [np.argmax(z) for y in predictions for x in y for z in x]))

              precision    recall  f1-score   support

           0       0.55      0.96      0.70      1923
           1       0.28      0.05      0.08       849
           2       0.00      0.00      0.00       748

    accuracy                           0.54      3520
   macro avg       0.28      0.34      0.26      3520
weighted avg       0.37      0.54      0.40      3520



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Unsurprisingly, our performance on the validation set is essentially equivalent to the majority baseline (i.e. guessing no structure for every amino acid). Now comes the fun part where we move to a real model and start pushing our performance. 

[array([[0.40791345, 0.48973104, 0.36919892],
        [0.6154315 , 0.49390164, 0.3681012 ],
        [0.68976194, 0.48868313, 0.33075163],
        [0.71436757, 0.48889843, 0.32723957],
        [0.73020184, 0.48290607, 0.30662486],
        [0.7349155 , 0.4789205 , 0.29619494],
        [0.7453747 , 0.42915016, 0.23457047],
        [0.7381913 , 0.4026277 , 0.21476215],
        [0.74038106, 0.3209078 , 0.17225957],
        [0.7167561 , 0.45103464, 0.2545771 ],
        [0.7312391 , 0.46839574, 0.28682798],
        [0.73136735, 0.48910987, 0.33612722],
        [0.7520437 , 0.45145422, 0.2624598 ],
        [0.7498765 , 0.36866084, 0.19953747],
        [0.70804375, 0.49472684, 0.36375278],
        [0.7562589 , 0.44054994, 0.25220197],
        [0.7256003 , 0.49143395, 0.34398603],
        [0.7373307 , 0.4943276 , 0.36230773],
        [0.76062965, 0.42870894, 0.23934396],
        [0.7462609 , 0.3704577 , 0.19891924],
        [0.72667813, 0.4457404 , 0.25251874],
        [0.74741983, 0.35227203, 0